In [36]:
import os 
import ee
import geemap
import numpy as np
import tqdm
import cv2

In [3]:
def scaled(image):  # for LANDSAT8 surfance reflectance
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

def select_bands(image): # select bands for training
    opticalBands = image.select('SR_B.')
    thermalRadianceBand = image.select('ST_TRAD')
    return image.addBands(opticalBands, None, True).addBands(thermalRadianceBand, None, True)

In [37]:
BOUNDING_BOX = [[[-95.690165, 29.5937], [-95.690165, 30.266005], [-94.900379, 30.266005], [-94.900379, 29.5937]]]

Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
polygon = ee.Geometry.Polygon(BOUNDING_BOX)
houston_bounds = ee.Geometry(polygon, None, False)
Map.addLayer(houston_bounds, {'color': 'FF0000'}, 'planar polygon');
Map

Map(center=[29.7662, -95.241], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [5]:
Map = geemap.Map()
# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# The collection contains images for multiple years and regions in the USA.
# print('Products:', dataset.aggregate_array('system:index'))

# Filter the collection to the 2016 product.
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()

# Each product has multiple bands for describing aspects of land cover.
# print('Bands:', nlcd2019.bandNames())

# Select the land cover band.
landcover_2019 = nlcd2019.select('landcover')

# Display land cover on the map.
Map.setCenter(-95, 38, 5)
Map.addLayer(landcover_2019, None, 'Landcover')
Map

Map(center=[38, -95], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [6]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
img = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20181205").select('ST_B10')
# img = ee.Image("LANDSAT/LC09/C02/T1_L2/LC09_025039_20220327")
geo = img.geometry()
Map.addLayer(img)
Map

Map(center=[29.7662, -95.241], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [7]:
Map = geemap.Map()
# image_props = geemap.image_props(img)
# print(image_props.getInfo())
geo = img.geometry()
Map.addLayer(landcover_2019.clip(geo), None, 'Landcover')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [35]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'nlcd2.tif')
# image = img.clip(geo).unmask()
geemap.ee_export_image(
    landcover_2019, filename=filename, scale=30, region=houston_bounds, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to /Users/yuhaoliu/Downloads/nlcd2.tif


In [36]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'landsat.tif')
# image = img.clip(geo).unmask()
geemap.ee_export_image(
    img.unmask(), filename=filename, scale=30, region=houston_bounds, file_per_band=False
)

The ee_object must be an ee.Image.


In [34]:
thermal = geemap.ee_to_numpy(img, region=houston_bounds)

Image.sampleRectangle: Too many pixels in sample; must be <= 262144. Got 6575848.


In [46]:
task = ee.batch.Export.image.toDrive(**{
    'image': img,
    'description': 'thermal_drive',
    'folder':'Example_folder',
    'scale': 30,
    'region': None  # houston_bounds.getInfo()['coordinates']
})
task.start()

In [50]:
task = ee.batch.Export.image.toDrive(**{
    'image': landcover_2019.clip(geo),
    'description': 'nlcd_drive4',
    'folder':'Example_folder',
    'scale': 30,
    'region': None
})
task.start()

In [45]:
task = ee.batch.Export.image.toDrive(**{
    'image': landcover_2019,
    'description': 'nlcd_drive2',
    'folder':'Example_folder',
    'scale': 30,
    'region': geo.getInfo()['coordinates']
})
task.start()

In [8]:
print(img.projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:32615', 'transform': [30, 0, 215685, 0, -30, 3471615]}


# try CRS

In [10]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
projection = img.projection().getInfo()
filename = os.path.join(out_dir, 'nlcd_crs.tif')
geemap.ee_export_image(
    landcover_2019, filename=filename, scale=30, region=houston_bounds, crs=projection['crs'], file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to /Users/yuhaoliu/Downloads/nlcd_crs.tif


In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
projection = img.projection().getInfo()
filename = os.path.join(out_dir, 'thermal_crs.tif')
geemap.ee_export_image(
    img, filename=filename, scale=30, region=houston_bounds, crs=projection['crs'], file_per_band=False
)

In [11]:
projection['crs']

'EPSG:32615'

# Do projections change?

In [35]:
from urllib.error import HTTPError
dates = ['20210807', '20210823', '20210908', '20210924', '20211010', '20211026', '20211111', '20211127', '20211213', '20211229', '20220114', '20220130', '20220215', '20220303', '20220319', '20220404', '20220420', '20220506', '20220522', '20220607']

for d in dates:
    try:
        img = ee.Image(f"LANDSAT/LC08/C02/T1_L2/LC08_025039_{d}").select('ST_B10')
        projection = img.projection().getInfo()
    except ee.EEException as e:
        print(e)
    if projection is not None:
        print(projection)

{'type': 'Projection', 'crs': 'EPSG:32615', 'transform': [30, 0, 215685, 0, -30, 3471615]}
{'type': 'Projection', 'crs': 'EPSG:32615', 'transform': [30, 0, 215985, 0, -30, 3471615]}
{'type': 'Projection', 'crs': 'EPSG:32615', 'transform': [30, 0, 215685, 0, -30, 3471615]}
{'type': 'Projection', 'crs': 'EPSG:32615', 'transform': [30, 0, 215685, 0, -30, 3471615]}
{'type': 'Projection', 'crs': 'EPSG:32615', 'transform': [30, 0, 213885, 0, -30, 3471615]}
{'type': 'Projection', 'crs': 'EPSG:32615', 'transform': [30, 0, 213585, 0, -30, 3471915]}
{'type': 'Projection', 'crs': 'EPSG:32615', 'transform': [30, 0, 215085, 0, -30, 3471615]}
Image.load: Image asset 'LANDSAT/LC08/C02/T1_L2/LC08_025039_20211127' not found.
{'type': 'Projection', 'crs': 'EPSG:32615', 'transform': [30, 0, 215085, 0, -30, 3471615]}
Image.load: Image asset 'LANDSAT/LC08/C02/T1_L2/LC08_025039_20211213' not found.
{'type': 'Projection', 'crs': 'EPSG:32615', 'transform': [30, 0, 215085, 0, -30, 3471615]}
Image.load: Image a

In [31]:
img0 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_025039_20190106')
img1 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_025039_20190122')
print(img)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.load",
    "arguments": {
      "id": {
        "constantValue": "LANDSAT/LC08/C02/T1_L2/LC08_025039_20190106"
      }
    }
  }
})
